# Inference & Entity Extraction (ETI) - Ungraph

Este notebook demuestra la fase **Inference** del patrón ETI (Extract-Transform-Inference) usando spaCy para extracción de entidades nombradas (NER).

## Objetivos

1. **Extracción de entidades** - Usar spaCy NER para extraer personas, organizaciones, lugares, etc.
2. **Extracción de relaciones** - Generar relaciones entre entidades co-ocurrentes
3. **Generación de facts** - Crear facts estructurados con trazabilidad
4. **Persistencia en el grafo** - Guardar entidades y facts en Neo4j
5. **Pipeline completo ETI** - Usar IngestDocumentUseCase con inferencia habilitada

## Requisitos

- `pip install ungraph[infer]`
- Modelo de spaCy: `python -m spacy download en_core_web_sm` (inglés)
- O: `python -m spacy download es_core_news_sm` (español)

**Referencias:**
- [Patrón ETI](../../docs/concepts/introduction.md)
- [SpacyInferenceService](../../src/infrastructure/services/spacy_inference_service.py)


In [ ]:
def add_src_to_path(path_folder: str):
    ''' 
    Helper function for adding the "path_folder" directory to the path.
    '''
    import sys
    from pathlib import Path

    base_path = Path().resolve()
    for parent in [base_path] + list(base_path.parents):
        candidate = parent / path_folder
        if candidate.exists():
            parent_dir = candidate.parent
            if str(parent_dir) not in sys.path:
                sys.path.insert(0, str(parent_dir))
            if str(candidate) not in sys.path:
                sys.path.append(str(candidate))
            return

# Agregar carpetas necesarias al path
add_src_to_path(path_folder="src")
add_src_to_path(path_folder="src/utils")
add_src_to_path(path_folder="src/data")


In [ ]:
# Importar librerías necesarias
import sys
from pathlib import Path

# Importar ungraph
try:
    import ungraph
    print("✅ Ungraph importado como paquete instalado")
except ImportError:
    import src
    ungraph = src
    print("✅ Ungraph importado desde src/ (modo desarrollo)")

# Importar servicios de inferencia
from infrastructure.services.spacy_inference_service import SpacyInferenceService
from domain.entities.chunk import Chunk
from application.dependencies import create_inference_service, create_ingest_document_use_case

print(f"📦 Ungraph version: {ungraph.__version__}")

# Verificar si spaCy está disponible
try:
    import spacy
    print("✅ spaCy está instalado")
    try:
        nlp = spacy.load("en_core_web_sm")
        print("✅ Modelo en_core_web_sm disponible")
    except OSError:
        print("⚠️  Modelo en_core_web_sm no encontrado. Instalar con: python -m spacy download en_core_web_sm")
except ImportError:
    print("❌ spaCy no está instalado. Instalar con: pip install spacy")


## Parte 1: Extracción de Entidades con spaCy

Primero, veamos cómo extraer entidades nombradas de un chunk de texto.


In [ ]:
# Crear servicio de inferencia
try:
    inference_service = SpacyInferenceService(model_name="en_core_web_sm")
    print("✅ SpacyInferenceService creado")
except Exception as e:
    print(f"❌ Error creando servicio: {e}")
    print("   Instalar con: pip install spacy && python -m spacy download en_core_web_sm")
    inference_service = None


In [ ]:
# Crear un chunk de ejemplo
sample_text = """
Apple Inc. is a technology company founded by Steve Jobs in Cupertino, California.
The company was established in 1976 and has since become one of the world's largest tech companies.
Tim Cook is the current CEO of Apple Inc.
"""

chunk = Chunk(
    id="chunk_example_1",
    page_content=sample_text,
    metadata={"filename": "example.txt", "page_number": 1}
)

print("📄 Chunk de ejemplo:")
print(sample_text)
print("\n" + "="*80)


In [ ]:
# Extraer entidades del chunk
if inference_service:
    print("🔍 Extrayendo entidades...\n")
    entities = inference_service.extract_entities(chunk)
    
    print(f"✅ Encontradas {len(entities)} entidades:\n")
    for i, entity in enumerate(entities, 1):
        print(f"{i}. {entity.name} ({entity.type})")
        print(f"   ID: {entity.id}")
        print(f"   Menciones: {entity.mentions}")
        print()
else:
    print("⚠️  Servicio de inferencia no disponible")


## Parte 2: Extracción de Relaciones

Las relaciones conectan entidades que aparecen juntas en el mismo chunk.


In [ ]:
# Extraer relaciones entre entidades
if inference_service and entities:
    print("🔗 Extrayendo relaciones...\n")
    relations = inference_service.extract_relations(chunk, entities)
    
    print(f"✅ Encontradas {len(relations)} relaciones:\n")
    for i, relation in enumerate(relations, 1):
        source_entity = next(e for e in entities if e.id == relation.source_entity_id)
        target_entity = next(e for e in entities if e.id == relation.target_entity_id)
        print(f"{i}. {source_entity.name} --[{relation.relation_type}]--> {target_entity.name}")
        print(f"   Confianza: {relation.confidence:.2f}")
        print(f"   Provenance: {relation.provenance_ref}")
        print()
else:
    print("⚠️  No hay entidades para extraer relaciones")


## Parte 3: Generación de Facts

Los facts son tripletas estructuradas (subject-predicate-object) con trazabilidad.


In [ ]:
# Generar facts desde el chunk
if inference_service:
    print("📊 Generando facts...\n")
    facts = inference_service.infer_facts(chunk)
    
    print(f"✅ Generados {len(facts)} facts:\n")
    for i, fact in enumerate(facts, 1):
        print(f"{i}. Tripleta: ({fact.subject}, {fact.predicate}, {fact.object})")
        print(f"   Confianza: {fact.confidence:.2f}")
        print(f"   Provenance: {fact.provenance_ref}")
        print(f"   Alta confianza: {fact.is_high_confidence()}")
        print()
else:
    print("⚠️  Servicio de inferencia no disponible")


## Parte 4: Pipeline Completo ETI con IngestDocumentUseCase

Ahora usemos el caso de uso completo con inferencia habilitada para ingerir un documento y extraer entidades automáticamente.


In [ ]:
# Crear caso de uso con inferencia habilitada
use_case = create_ingest_document_use_case(
    enable_inference=True,
    inference_language="en"  # "en" para inglés, "es" para español
)

print("✅ IngestDocumentUseCase creado con inferencia habilitada")
print("   El pipeline ejecutará: Extract → Transform → Inference")


In [ ]:
# Buscar un archivo de ejemplo para ingerir
from src.utils.handlers import find_in_project

# Buscar archivos de ejemplo
example_files = find_in_project("*.md", "src/data")
if example_files:
    example_file = example_files[0]
    print(f"📄 Archivo encontrado: {example_file}")
    print(f"   Usaremos este archivo para demostrar el pipeline ETI completo\n")
else:
    print("⚠️  No se encontraron archivos de ejemplo")
    print("   Puedes crear un archivo de texto o usar uno existente")
    example_file = None


In [ ]:
# Ejecutar pipeline ETI completo
if example_file and use_case:
    print("🚀 Ejecutando pipeline ETI completo...\n")
    print("   Fases:")
    print("   1. Extract: Cargar documento")
    print("   2. Transform: Dividir en chunks, generar embeddings")
    print("   3. Inference: Extraer entidades, relaciones y facts")
    print("   4. Persistir: Guardar chunks y facts en el grafo\n")
    
    try:
        chunks = use_case.execute(
            file_path=Path(example_file),
            chunk_size=500,
            chunk_overlap=100
        )
        
        print(f"✅ Pipeline completado:")
        print(f"   - Chunks creados: {len(chunks)}")
        print(f"   - Entidades y facts extraídos y persistidos en el grafo")
        
    except Exception as e:
        print(f"❌ Error ejecutando pipeline: {e}")
        import traceback
        traceback.print_exc()
else:
    print("⚠️  No se puede ejecutar el pipeline (archivo o use_case no disponible)")


## Parte 5: Consultar Entidades y Facts en el Grafo

Ahora consultemos las entidades y facts que fueron persistidos en Neo4j.


In [ ]:
# Consultar entidades en el grafo
from src.utils.graph_operations import graph_session

driver = graph_session()
try:
    with driver.session() as session:
        # Contar entidades
        result = session.run("MATCH (e:Entity) RETURN count(e) as count")
        entity_count = result.single()["count"]
        print(f"📊 Entidades en el grafo: {entity_count}")
        
        # Mostrar algunas entidades
        if entity_count > 0:
            result = session.run("""
                MATCH (e:Entity)
                RETURN e.name as name, e.type as type, size(e.mentions) as mention_count
                ORDER BY mention_count DESC
                LIMIT 10
            """)
            
            print("\n🔝 Top 10 entidades por menciones:")
            for record in result:
                print(f"  - {record['name']} ({record['type']}) - {record['mention_count']} menciones")
finally:
    driver.close()


In [ ]:
# Consultar facts en el grafo
driver = graph_session()
try:
    with driver.session() as session:
        # Contar facts
        result = session.run("MATCH (f:Fact) RETURN count(f) as count")
        fact_count = result.single()["count"]
        print(f"📊 Facts en el grafo: {fact_count}")
        
        # Mostrar algunos facts
        if fact_count > 0:
            result = session.run("""
                MATCH (f:Fact)
                RETURN f.subject as subject, f.predicate as predicate, f.object as object, f.confidence as confidence
                ORDER BY f.confidence DESC
                LIMIT 10
            """)
            
            print("\n🔝 Top 10 facts por confianza:")
            for record in result:
                print(f"  - ({record['subject']}, {record['predicate']}, {record['object']})")
                print(f"    Confianza: {record['confidence']:.2f}")
finally:
    driver.close()


## Parte 6: Visualización de Entidades con spaCy

spaCy incluye herramientas de visualización para mostrar entidades extraídas.


In [ ]:
# Visualizar entidades con spaCy displacy
try:
    import spacy
    from spacy import displacy
    
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(sample_text)
    
    # Visualizar entidades (solo funciona en Jupyter)
    print("📊 Visualización de entidades (ejecutar en Jupyter para ver visualización):")
    displacy.render(doc, style="ent", jupyter=True)
    
except Exception as e:
    print(f"⚠️  No se puede mostrar visualización: {e}")
    print("   Esto es normal si no estás ejecutando en Jupyter")


## Resumen y Mejores Prácticas

### Tipos de Entidades Extraídas

spaCy puede extraer diferentes tipos de entidades:
- **PERSON**: Personas (ej: "Steve Jobs", "Tim Cook")
- **ORGANIZATION**: Organizaciones (ej: "Apple Inc.")
- **LOCATION/GPE**: Lugares (ej: "Cupertino", "California")
- **DATE**: Fechas (ej: "1976")
- **MONEY**: Cantidades monetarias
- **PERCENT**: Porcentajes
- Y más...

### Flujo del Pipeline ETI

1. **Extract**: Cargar documento desde archivo
2. **Transform**: 
   - Dividir en chunks
   - Generar embeddings
   - Persistir chunks en el grafo
3. **Inference**:
   - Extraer entidades nombradas (NER)
   - Generar relaciones entre entidades
   - Crear facts estructurados
   - Persistir entidades y facts en el grafo

### Configuración de Idiomas

Para español:
```python
use_case = create_ingest_document_use_case(
    enable_inference=True,
    inference_language="es"  # Requiere: python -m spacy download es_core_news_sm
)
```

### Mejores Prácticas

1. **Instalar modelos apropiados**: Descarga el modelo de spaCy para tu idioma
2. **Ajustar chunk_size**: Chunks más pequeños pueden mejorar la precisión de NER
3. **Revisar confianza**: Los facts tienen niveles de confianza que puedes filtrar
4. **Trazabilidad**: Todos los facts tienen `provenance_ref` al chunk origen

## Referencias

- [Patrón ETI](../../docs/concepts/introduction.md)
- [SpacyInferenceService](../../src/infrastructure/services/spacy_inference_service.py)
- [spaCy Documentation](https://spacy.io/)
el 